In [1]:
# Gets historical weather data from WunderGround
# Using BeautifulSoup and Selenium to get the monthly weather stats

import requests, sys, re

from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.keys import Keys


In [2]:
def get_page(html, attempt=0, threshold=10):
    page = requests.get(html)
    try: 
        assert(199 < page.status_code < 300)
        return page
    except AssertionError:
        if attempt < threshold:
            return get_page(html, attempt + 1, threshold=threshold)
        else:
            raise Exception('Exceeded maximum attempts')

In [3]:
url = 'https://www.wunderground.com/history/monthly/KLGA/date/2020-3'

bi = FirefoxBinary(r'C:\Program Files (x86)\Mozilla Firefox\\firefox.exe')
br = webdriver.Firefox(firefox_binary=bi)

br.get(url)

# soup = bs(br.page_source, 'html.parser')
soup = bs(br.page_source, 'lxml')

br.quit()

tables = soup.find_all('table')

In [7]:
# Write all the tables into csv files:
for i in range(len(tables)):
    out_file = open('wunderground' + str(i + 1) + '.csv', 'w')
    table = tables[i]

    # ---- Write the table header: ----
    table_head = table.findAll('th')
    output_head = []
    for head in table_head:
        output_head.append(head.text.strip())

    # Some cleaning and formatting of the text before writing:
    header = '"' + '";"'.join(output_head) + '"'
    header = re.sub('\s', '', header) + '\n'
    out_file.write(header)

    # ---- Write the rows: ----
    output_rows = []
    rows = table.findAll('tr')
    for j in range(1, len(rows)):
        table_row = rows[j]
        columns = table_row.findAll('td')
        output_row = []
        for column in columns:
            output_row.append(column.text.strip())

        # Some cleaning and formatting of the text before writing:
        fila = '"' + '";"'.join(output_row) + '"'
        fila = re.sub('\s', '', fila) + '\n'
        out_file.write(fila)

    out_file.close()